In [12]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
#base64 needed to be imported.
import base64

#Changed animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from MongoCreateRead import AnimalShelter





###########################
# Data Manipulation / Model
###########################
#Changed for your username and password and CRUD Python module name
#Also changed parameters of CRUD Python module to conform to requirements I set in my design.
username = "aacuser"
password = "456password"
# NOTE: I set up my AnimalShelter class to take port and database info as well to make the module more resusable with other mongo databases.
shelter = AnimalShelter(55723, 'AAC', username, password)


# class read method must support return of cursor object
# NOTE: Adjusted call to read function to match my CRUD Python modules requirements as it has a slightly different name and parameter requirements.
# In order to increase future usablity of the module with other mongo databases,
# it takes the collection name to use as a parameter.
query = {}
df = pd.DataFrame.from_records(shelter.readQuery('animals', query))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project Two')


# Added in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replaced with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#DONE Place the HTML image tag in the line below into the app.layout code according to your design
#DONE Also remember to include a unique identifier such as your name or date

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    #Image tag to display teh GrazioSalvareLogo
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
#DONE Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
#TODO: Maybe some formatting to better position the buttons.  Would be nice to get the button and label for Reset to all be on the same line at least.
        #Using RadioItems (radio buttons), to select which filter is active as it makes sense that only one may be selected at a time and defaulting to the reset option.
        dcc.RadioItems(
            id='filter-type',
            labelStyle={'display': 'block'},
            options=[
                {'label': 'Water Rescue', 'value': 1},
                {'label': 'Mountain or Wilderness Rescue', 'value': 2},
                {'label': 'Disaster or Individual Tracking', 'value': 3},
                {'label': 'Reset/Unfiltered', 'value': 0}
            ],
            value=0,
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#DONE: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        #Table feature settings.
        editable=False, 
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
        #Hiding a few of the columns that are likely to be of low value, we have an age column and we can look up the date on our own watch.
        #Also adding a bit of extra styling options to make the table more readable.
        style_cell_conditional=[
            {'if': {'column_id': 'date_of_birth'},
                'display': 'None'},
            {'if': {'column_id': 'datetime'},
                'display': 'None'},
            {'if': {'column_id': 'monthyear'},
                'display': 'None'},
            {'if': {'column_id': 'age_upon_outcome'},
                'display': 'None'},
            {'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(220, 220, 220)'},
        ],
        style_cell={'padding': '5px'},
        style_header={'fontWeight': 'bold'},
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    #Unique identifier code
    html.P("Tim Gallus SNHU CS-340-H7516 Client/Server Development")
])
#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### DONE: Add code to filter interactive data table with MongoDB queries
#Using if-elif-else creating a query to use based on the filter-type that was selected in our radio buttons. 
    if filter_type == 0:
        query = {}
    elif filter_type == 1:                 
        query = {"$and": [{"sex_upon_outcome": "Intact Female"}, {"$or": [ 
            {"breed": "Labrador Retriever Mix"},
            {"breed": "Chesapeake Bay Retriever"},
            {"breed": "Newfoundland"}]},
            {"$and": [{"age_upon_outcome_in_weeks": {"$gte": 26}}, {"age_upon_outcome_in_weeks": {"$lte": 156}}]
            }]}
    elif filter_type == 2:
        query = {"$and": [{"sex_upon_outcome": "Intact Male"}, {"$or": [ 
            {"breed": "German Shepard"},
            {"breed": "Alaskan Malamute"},
            {"breed": "Old English Sheepdog"},
            {"breed": "Siberian Husky"},
            {"breed": "Rottweiler"}]},
            {"$and": [{"age_upon_outcome_in_weeks": {"$gte": 26}}, {"age_upon_outcome_in_weeks": {"$lte": 156}}]
            }]}
    elif filter_type == 3:
        query = {"$and": [{"sex_upon_outcome": "Intact Male"}, {"$or": [ 
            {"breed": "Doberman Pinscher"},
            {"breed": "German Shepard"},
            {"breed": "Golden Retriever"},
            {"breed": "Bloodhound"},
            {"breed": "Rottweiler"}]},
            {"$and": [{"age_upon_outcome_in_weeks": {"$gte": 20}}, {"age_upon_outcome_in_weeks": {"$lte": 300}}]
            }]}
    else:
        query = {}
    
    #Adjusted database readQuery call to match what my python module uses and to take the query we created based on the radio button selection as a parameter (along with the collection parameter).
    df = pd.DataFrame.from_records(shelter.readQuery('animals', query))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    #DONE
    # added code for pie chart #
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame = dff,
                values = dff["breed"].value_counts(),
                names = dff["breed"].value_counts().keys()
            )
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', 'selected_rows'),
    Input('datatable-id', 'selected_columns')])
def update_map(viewData, selected_rows, selected_columns):
#DONE: Add in the code for your geolocation chart
#Based on Code from Module Six.
#Code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30,-97], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            #Placing Marker on the map based on latitude and longitude column in the datatable.
            #Includes a tooltip based on animal type with a pop up that has the animals name.
            dl.Marker(position=[dff.iloc[selected_rows[0], 13], dff.iloc[selected_rows[0], 14]], children=[  
                dl.Tooltip(dff.iloc[selected_rows[0], 3]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[selected_rows[0], 9])
                ])
            ])
        ])
    ]



app